In [3]:
%cd /Users/maayanlab/Documents/projects/human-melanoma-signature
import sys
sys.path.append('pipeline/scripts/')
import pandas as pd
import h5py
import numpy as np
import json, operator, math, requests
from geode_jupies import *
import sklearn.metrics.pairwise as smp
from scipy.spatial.distance import cosine
import seaborn as sns; sns.set(color_codes=True)
import RNAseq as RNA
from collections import Counter
import time

/Users/maayanlab/Documents/projects/human-melanoma-signature


/Users/maayanlab/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
%pwd

'/Users/maayanlab/Documents/projects/human-melanoma-signature'

In [4]:
from Melanoma import *

# Code Development for Melanoma Pipeline
## Define input and output files

In [51]:
# infiles S4.4
#infile = 's4-enrichment.dir/melanoma-enrichr-results.txt'
#infile2 = 's4-enrichment.dir/enrichr-results-top.txt'

In [22]:
# infiles
infile1 = 's2-signatures.dir/melanoma-intersection-2000.txt'
infile2 = 's2-signatures.dir/melanoma-intersection-3000.txt'
infile3 = 's2-signatures.dir/melanoma-intersection-5000.txt'

In [166]:
# infiles S2.2
sig = 's2-signatures.dir/melanoma-signatures.txt'
outfile = 's2-signatures.dir/new-melanoma-signatures-top50-union.txt'

In [5]:
# #infiles
# cpm = 's1-expression_data.dir/melanoma-cpm.txt'
# matches = 's2-expression_data.dir/melanoma-idmatches.txt'

In [7]:
# # Infile & outfile
# infiles = ['s1-expression_data.dir/melanoma-cpm.txt']
# outfile = 's1-expression_data.dir/melanoma-IDstatus.txt'

In [8]:
# # Infile & outfile
# infiles = ['rawdata/human_matrix.h5', 'rawdata/m2.csv']
# outfile = 's1-expression_data.dir/melanoma-cpm.txt'

### Test Code

In [125]:
sig_df=pd.read_table(sig, index_col='gene')
sig_df

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,2.221160e-04,-0.000741,-0.000528,-0.000293,0.000081,-0.000679,-0.000913,-0.000520,0.000217,-0.000439,...,-0.001003,-7.091145e-04,-0.000192,-0.000019,-0.000432,-0.000729,-0.000202,0.001092,0.000437,1.121692e-04
A2M,6.466985e-03,-0.000725,-0.002951,-0.003329,-0.002101,-0.024287,-0.024964,-0.002382,-0.000219,-0.001720,...,0.000173,1.450300e-04,-0.000058,-0.008256,0.273626,0.135737,0.041137,0.000944,-0.019554,-9.802424e-04
A2ML1,-1.235817e-04,-0.000725,-0.000793,-0.000344,0.000104,-0.000761,-0.000908,-0.000830,0.000205,-0.000298,...,0.000173,-5.797044e-05,0.000139,0.000249,-0.000923,-0.001315,-0.000556,0.001044,0.000681,1.505726e-05
A2MP1,-1.037960e-04,-0.000750,-0.000818,-0.000365,0.000101,-0.000775,-0.000919,-0.000840,0.000208,-0.000287,...,0.000162,-5.557867e-05,0.000130,0.000238,-0.000903,-0.001314,-0.000528,0.001051,0.000619,1.639789e-05
A4GALT,7.294164e-04,-0.000740,-0.000195,-0.001962,-0.000815,-0.000589,-0.000910,-0.000797,0.000213,-0.000504,...,0.000122,-1.564616e-04,0.000164,0.000202,-0.000940,-0.001320,0.002802,0.001034,0.000665,-3.905606e-04
AAAS,-2.887554e-05,-0.000671,0.000764,0.000602,-0.001139,0.000240,0.000240,0.000656,-0.000428,-0.002003,...,0.000154,1.891351e-05,0.000084,0.000367,0.000681,0.000030,0.000057,-0.000884,0.000906,-4.390309e-04
AACS,2.675834e-04,0.005065,-0.000697,-0.000205,-0.000929,-0.000864,-0.000125,-0.000464,-0.000521,0.003434,...,0.001846,7.199311e-04,0.002248,-0.000994,-0.000186,-0.001993,-0.001034,0.000374,-0.000285,4.808830e-04
AACSP1,-9.101854e-05,-0.000709,-0.000794,-0.000353,0.000102,-0.000758,-0.000902,-0.000758,0.000169,-0.000357,...,0.000162,-5.557867e-05,0.000130,0.000544,-0.000930,-0.001318,-0.000554,0.001047,0.000664,1.668024e-05
AADACP1,-1.113141e-04,-0.000780,-0.000779,-0.000362,0.000115,-0.000758,-0.000903,-0.000831,0.000198,-0.000299,...,0.000160,-5.714934e-05,0.000130,0.000273,-0.000924,-0.001313,-0.000557,0.001049,0.000666,1.246117e-05


In [94]:
# Make each signature to a np.array
s17a=sig_df['17_a'].values
s17b=sig_df['17_b'].values
s17c=sig_df['17_c'].values
s2a=sig_df['2_a'].values
s2b=sig_df['2_b'].values
s2c=sig_df['2_c'].values
s15a=sig_df['15_a'].values
s15b=sig_df['15_b'].values
s15c=sig_df['15_c'].values
s18a=sig_df['18_a'].values
s18b=sig_df['18_b'].values
s18c=sig_df['18_c'].values
s19a=sig_df['19_a'].values
s19b=sig_df['19_b'].values
s19c=sig_df['19_c'].values
s19d=sig_df['19_d'].values
s1=sig_df['1'].values
s5a=sig_df['5_a'].values
s5b=sig_df['5_b'].values
s5c=sig_df['5_c'].values
s11=sig_df['11'].values
s3=sig_df['3'].values
# Take vector sum per cluster as a representation of intersection
cluster1_inter= s17a + s17b + s17c + s2a + s2b + s2c
cluster2_inter= s15c + s19b + s19d
cluster3_inter= s15a + s15b + s3 + s18b + s11 + s19a
cluster4_inter= s18c + s5b +s5c
cluster5_inter= s1 + s19c
cluster6_inter= s18a + s5a

In [126]:
# Sort the array in reserve order
sort_idx_1 = np.argsort(np.abs(cluster1_inter))[::-1]
sort_idx_2 = np.argsort(np.abs(cluster2_inter))[::-1]
sort_idx_3 = np.argsort(np.abs(cluster3_inter))[::-1]
sort_idx_4 = np.argsort(np.abs(cluster4_inter))[::-1]
sort_idx_5 = np.argsort(np.abs(cluster5_inter))[::-1]
sort_idx_6 = np.argsort(np.abs(cluster6_inter))[::-1]

In [123]:
#sorted_genes_1 = sig_df['gene'][sort_idx_1][:50].drop(index=None, columns='gene')

In [167]:
# Return an index of top 5000 summed genes for this cluster
sorted_genes_1 = sig_df.index[sort_idx_1][:50]
sorted_genes_2 = sig_df.index[sort_idx_2][:50]
sorted_genes_3 = sig_df.index[sort_idx_3][:50]
sorted_genes_4 = sig_df.index[sort_idx_4][:50]
sorted_genes_5 = sig_df.index[sort_idx_5][:50]
sorted_genes_6 = sig_df.index[sort_idx_6][:50]

In [168]:
export= set(sorted_genes_1) | set(sorted_genes_2) | set(sorted_genes_3) | set(sorted_genes_4) | set(sorted_genes_5) | set(sorted_genes_6)

In [169]:
test1=sig_df.loc[export]

In [170]:
test1

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
gene,,,,,,,,,,,,,,,,,,,,,
TPM4,-0.057179,0.014896,0.016461,-0.002860,0.015418,-0.005739,0.019065,-0.003221,-0.000897,0.011827,...,-0.026223,0.000545,-0.031414,0.001101,0.001916,0.034337,0.024049,-0.093785,-0.017269,-1.193901e-02
IGFBP5,0.000316,-0.000821,-0.000822,-0.000325,0.000051,-0.000923,-0.000958,0.012091,0.000441,-0.035680,...,0.000146,0.005996,-0.006567,-0.000045,-0.000912,-0.001317,0.020807,-0.005015,0.010946,6.540801e-01
MT-CO1,0.044458,0.007475,0.097969,0.108239,-0.046051,-0.214850,-0.300970,-0.218844,0.033958,-0.202263,...,0.011695,0.005166,0.014559,0.232724,0.149029,0.161010,0.221268,0.310243,-0.127187,-5.807148e-02
DCT,0.000052,-0.000738,-0.000799,-0.000352,0.000102,-0.075731,-0.124897,-0.115012,0.000189,-0.003240,...,0.000162,-0.000056,0.000130,-0.109246,-0.002155,-0.002932,-0.000548,-0.004324,-0.169040,-5.741516e-01
CD63,-0.003033,0.009523,0.013134,-0.003645,-0.005263,0.020560,-0.034753,-0.002619,-0.004782,0.003835,...,-0.107659,-0.052226,-0.142066,-0.051926,-0.024105,-0.040364,0.005264,-0.047726,-0.049897,-2.154445e-02
PDIA3,-0.015514,0.003842,0.015899,0.012983,-0.022073,0.013117,0.017840,0.019867,0.014637,0.159988,...,-0.003415,0.040123,-0.038536,-0.018753,-0.002760,-0.012736,0.037756,-0.020195,-0.060201,-7.673429e-02
HLA-B,0.063029,0.000499,-0.007926,-0.026376,0.013222,-0.002088,0.002202,0.000048,-0.003053,-0.015077,...,-0.003625,0.007351,-0.008496,-0.000732,0.023847,0.034137,0.029462,-0.007207,0.001009,9.532065e-05
GNAS,-0.001556,0.005035,0.021324,0.006424,-0.013480,0.042624,-0.006068,0.032853,0.002464,-0.054028,...,0.001419,0.003830,-0.004050,-0.012502,0.024183,0.011516,-0.444624,-0.031456,-0.033607,-8.700574e-03
MDM2,0.000301,0.010822,-0.002552,0.000257,0.003334,0.001115,0.001935,0.006729,0.001876,0.002434,...,0.023692,0.146962,-0.056711,0.000587,0.014778,0.018533,-0.003985,0.007102,0.002293,1.485862e-02


In [171]:
test1.to_csv(outfile, sep='\t')

In [60]:
union_sig=sig_df[sig_df['gene'].isin(export)].set_index('gene')

,1,11,15_a,15_b,15_c,17_a,17_b,17_c,18_a,18_b,...,19_b,19_c,19_d,2_a,2_b,2_c,3,5_a,5_b,5_c
gene,,,,,,,,,,,,,,,,,,,,,
A2M,0.006467,-0.000725,-0.002951,-0.003329,-0.002101,-0.024287,-0.024964,-0.002382,-0.000219,-0.001720,...,0.000173,0.000145,-0.000058,-0.008256,0.273626,0.135737,0.041137,0.000944,-0.019554,-0.000980
ABCA1,-0.003329,-0.000226,-0.001765,0.000096,0.001180,-0.001071,-0.001124,-0.001089,0.000037,-0.000159,...,0.000020,-0.000823,0.001151,0.000246,-0.000808,0.001602,0.000046,0.019761,0.003275,0.004170
AC010970_2,0.000788,0.229345,-0.031733,-0.013356,-0.012395,0.001140,0.000619,0.005694,-0.030024,0.000078,...,0.003295,0.001705,0.004977,0.000043,0.125056,0.091808,-0.213738,-0.116451,0.087223,0.024547
ACTB,0.037596,-0.011550,0.031596,-0.070839,-0.066182,0.078543,0.078871,0.048800,-0.070905,0.032163,...,0.011890,0.014489,-0.016220,0.066613,0.081459,0.117495,0.018404,-0.365970,-0.033042,-0.098589
ACTG1,0.097909,0.007218,0.004113,-0.072776,-0.000414,0.057116,0.044939,-0.007710,-0.060800,0.022915,...,0.149535,0.089686,-0.102638,-0.019425,0.061176,0.080928,0.094846,-0.179033,-0.077832,-0.082254
ACTN4,0.025991,0.017905,-0.001552,-0.004419,0.003380,0.034166,0.038817,0.031361,-0.015434,0.024284,...,-0.010127,-0.008663,-0.005308,0.004601,0.028090,0.028267,-0.006412,-0.084071,-0.006996,-0.008208
AHNAK2,0.004171,-0.001105,-0.002552,-0.000858,0.000838,-0.000792,0.007685,0.002075,-0.000075,0.000605,...,-0.000941,-0.001249,-0.000444,-0.000012,-0.001011,-0.001521,0.002465,0.018167,0.000238,0.000211
AKAP12,-0.000900,-0.013313,0.098115,0.059234,-0.149381,0.008609,0.018395,0.006287,-0.001824,0.002994,...,-0.005118,-0.014056,0.003596,-0.008003,-0.002065,0.000160,0.047743,0.010101,-0.011742,-0.004996
ALDH1A1,-0.000109,-0.000752,0.163501,0.142334,-0.166399,0.004588,-0.001088,-0.000196,0.000193,-0.000346,...,0.000162,0.000086,0.000033,0.000285,-0.000983,-0.001128,-0.000590,0.000930,0.005934,0.000001


In [63]:
union_sig.to_csv(outfile, sep='\t')

In [65]:
from IPython.display import HTML, display
# to display hyperlink as <a> tag in output cells
def display_link(url):
    raw_html = '<a href="%s" target="_blank">%s</a>' % (url, url)
    return display(HTML(raw_html))

In [66]:
clustergrammer_url = 'http://amp.pharm.mssm.edu/clustergrammer/matrix_upload/'
r = requests.post(clustergrammer_url, files={'file': open(outfile, 'rb')})
link = r.text
display_link(link)

In [67]:
from IPython.display import IFrame
display(IFrame(link, width="1000", height="1000"))

In [24]:
top2000_df = pd.read_table(infile1)

In [19]:
genes = top2000_df['gene'].values

In [20]:
result_df=run_enrichr(genes)

In [27]:
sig_df = pd.read_table(sig).set_index('gene')
# Make each signature to a np.array
s17a=sig_df['17_a'].values
s17b=sig_df['17_b'].values
s17c=sig_df['17_c'].values
s2a=sig_df['2_a'].values
s2b=sig_df['2_b'].values
s2c=sig_df['2_c'].values
s15a=sig_df['15_a'].values
s15b=sig_df['15_b'].values
s15c=sig_df['15_c'].values
s18a=sig_df['18_a'].values
s18b=sig_df['18_b'].values
s18c=sig_df['18_c'].values
s19a=sig_df['19_a'].values
s19b=sig_df['19_b'].values
s19c=sig_df['19_c'].values
s19d=sig_df['19_d'].values
s1=sig_df['1'].values
s5a=sig_df['5_a'].values
s5b=sig_df['5_b'].values
s5c=sig_df['5_c'].values
s11=sig_df['11'].values
s3=sig_df['3'].values
# Take vector sum per cluster as a representation of intersection
cluster1_inter= s17a + s17b + s17c + s2a + s2b + s2c
cluster2_inter= s15c + s19b + s19d
cluster3_inter= s15a + s15b + s3 + s18b + s11
cluster4_inter= s18c + s5b +s5c
cluster5_inter= s1 + s19c
cluster6_inter= s18a + s5a
# Sort the array in reserve order
sort_idx_1 = np.argsort(cluster1_inter)[::-1]
sort_idx_2 = np.argsort(cluster2_inter)[::-1]
sort_idx_3 = np.argsort(cluster3_inter)[::-1]
sort_idx_4 = np.argsort(cluster4_inter)[::-1]
sort_idx_5 = np.argsort(cluster5_inter)[::-1]
sort_idx_6 = np.argsort(cluster6_inter)[::-1]

In [40]:
# Return an index of top 5000 summed genes for this cluster
sorted_genes_1 = sig_df.index[sort_idx_1][:50]
sorted_genes_2 = sig_df.index[sort_idx_2][:50]
sorted_genes_3 = sig_df.index[sort_idx_3][:50]
sorted_genes_4 = sig_df.index[sort_idx_4][:50]
sorted_genes_5 = sig_df.index[sort_idx_5][:50]
sorted_genes_6 = sig_df.index[sort_idx_6][:50]

In [56]:
# Intersection of all 6 clusters's top 500 differentially exp. genes
export_df=pd.DataFrame (sorted_genes_1 | sorted_genes_2 | sorted_genes_3 | sorted_genes_4 | sorted_genes_5 | sorted_genes_6)

In [57]:
export=sorted_genes_1 | sorted_genes_2 | sorted_genes_3 | sorted_genes_4 | sorted_genes_5 | sorted_genes_6
export

Index(['A2M', 'ABCA1', 'AC010970_2', 'ACTB', 'ACTG1', 'ACTN4', 'AHNAK2',
       'AKAP12', 'ALDH1A1', 'ALDOA',
       ...
       'TXNIP', 'TXNRD1', 'UBC', 'VCAN', 'VIM', 'VWF', 'WWTR1', 'YBX1',
       'YWHAZ', 'ZNF3'],
      dtype='object', name='gene', length=230)

In [35]:
export.to_csv(outfile, sep='\t', index=False)

In [43]:
genes = export['gene']

In [44]:
result_df = run_enrichr(genes)

In [45]:
result_df

,term_name,zscore,combined_score,FDR,pvalue,overlapping_genes,gene_set_library
0,protein targeting to ER (GO:0045047),-2.085357,91.031606,5.322236e-17,1.101143e-19,"[RPL4, RPL41, RPL3, RPL10, RPS8, RPL23, RPLP1,...",GO_Biological_Process_2018
1,SRP-dependent cotranslational protein targetin...,-1.840410,83.694450,1.773965e-17,1.778411e-20,"[RPL4, RPL41, RPL3, RPL10, RPS8, RPL23, RPLP1,...",GO_Biological_Process_2018
2,"nuclear-transcribed mRNA catabolic process, no...",-1.997175,81.207360,7.292814e-16,2.193327e-18,"[RPL4, RPL41, RPL3, RPL10, RPS8, RPL23, RPLP1,...",GO_Biological_Process_2018
3,extracellular matrix organization (GO:0030198),-1.641114,71.324472,5.322236e-17,1.333894e-19,"[ITGB1, SPARC, COL11A1, SERPINE1, TNC, LAMC1, ...",GO_Biological_Process_2018
4,peptide biosynthetic process (GO:0043043),-1.914538,70.881497,1.848915e-14,8.340968e-17,"[RPL4, PCSK1, RPL41, RPL3, RPL10, RPS8, RPL23,...",GO_Biological_Process_2018
5,viral transcription (GO:0019083),-1.783768,66.887449,1.293451e-14,5.186770e-17,"[RPL4, RPL41, RPL3, RPL10, RPS8, RPL23, RPLP1,...",GO_Biological_Process_2018
6,cellular protein metabolic process (GO:0044267),-1.179625,65.464684,1.578545e-21,7.912506e-25,"[RPL4, RPL3, RPL10, RPLP1, RPLP0, TNC, LAMC1, ...",GO_Biological_Process_2018
7,regulated exocytosis (GO:0045055),-2.282059,62.150413,2.118757e-10,1.486847e-12,"[SERPINA3, SPARC, VWF, APLP2, SERPINE1, FN1, A...",GO_Biological_Process_2018
8,cotranslational protein targeting to membrane ...,-1.368407,60.951054,3.010519e-17,4.527096e-20,"[RPL4, RPL41, RPL3, RPL10, RPS8, RPL23, RPLP1,...",GO_Biological_Process_2018
9,viral gene expression (GO:0019080),-1.577981,59.994607,8.770762e-15,3.077461e-17,"[RPL4, RPL41, RPL3, RPL10, RPS8, RPL23, RPLP1,...",GO_Biological_Process_2018


In [234]:
c = Counter()
for i in [enr_res[signature]['top'] for signature in enr_res.keys()]:
    c.update(i['term_name'].iloc[:5])

In [235]:
c.most_common()

[('protein targeting to ER (GO:0045047)', 17),
 ('SRP-dependent cotranslational protein targeting to membrane (GO:0006614)',
  17),
 ('nuclear-transcribed mRNA catabolic process, nonsense-mediated decay (GO:0000184)',
  17),
 ('viral transcription (GO:0019083)', 13),
 ('peptide biosynthetic process (GO:0043043)', 8),
 ('viral process (GO:0016032)', 5),
 ('neutrophil degranulation (GO:0043312)', 4),
 ('extracellular matrix organization (GO:0030198)', 3),
 ('neutrophil mediated immunity (GO:0002446)', 3),
 ('regulated exocytosis (GO:0045055)', 2),
 ('platelet degranulation (GO:0002576)', 2),
 ('rRNA metabolic process (GO:0016072)', 2),
 ('mitochondrial ATP synthesis coupled proton transport (GO:0042776)', 1),
 ('ribosomal small subunit export from nucleus (GO:0000056)', 1),
 ('positive regulation of nitric oxide biosynthetic process (GO:0045429)', 1),
 ('positive regulation of nitric oxide metabolic process (GO:1904407)', 1),
 ('ER to Golgi vesicle-mediated transport (GO:0006888)', 1),
 

In [ ]:
c = Counter()
for i in [results[signature]['top'] for signature in results.keys()]
for i in [results['1']['top']]: 
    c.update(i['Term name'].iloc[:5])
[results[signature]['top'] for signature in results.keys()]

In [176]:
# Create a 3rd dictionary 
from collections import defaultdict
results = defaultdict(dict)
# Iterate through dictionary, create gene lists and save as 'genes'
for signature in dict_top_genes:
    for direction in ('top', 'bottom'):
        genes = dict_top_genes[signature][direction] # value = dict [1st key] [2nd key]
        results[signature][direction] = enrichr_result_to_df(genes,'','GO_Biological_Process_2018')
        #for signature in list(dict_top_genes.keys())[0]:

# freeze default dict for readonly1
results.default_factory = None

In [201]:
# Number of studies to be processed
n = len(study_names)
i = 0
for study in study_names[:]:
    
df_1_top=pd.DataFrame(results['1']['top'])

In [209]:
# if creating a list instead of dictionary
test = []
for signature in dict_top_genes:
    for direction in ('top', 'bottom'):
        genes = dict_top_genes[signature][direction] # value = dict [1st key] [2nd key]
        genes_df = pd.DataFrame(enrichr_result_to_df(genes,'','GO_Biological_Process_2018'))
        #specify names of each geneset (which study and top/bottom they are from)
        test.append([signature, direction, genes_df])

In [ ]:
for study, d1 in D.items():
    for top, v in d1.items():
wanted_keys = dict_df.columns.get_values().tolist() # The keys you want
dict((k, results[k]) for k in wanted_keys if k in results)    